In [1]:
# Importing necessary libraries
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io

In [2]:
# Path to the zip file containing the dataset
# Downloading and extracting the Spanish-English translation dataset
path_to_zip = keras.utils.get_file('spa-eng.zip', origin='https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip', extract=True) # dataset text with its translation


In [3]:
# Define the path to the extracted file
path_to_file = os.path.join(os.path.dirname(path_to_zip), 'spa-eng', 'spa.txt') # get file of dataset


In [4]:
def unicode_to_ascii(s): # Function to normalize Unicode characters
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [5]:
def preprocess_sentence(w): # Function to preprocess a sentence
    w = unicode_to_ascii(w.lower().strip()) # Convert to lowercase and normalize
    w = re.sub(r"([?.!,])", r" \1 ", w) # Add spaces before punctuation
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.~,]+", " ", w)# Remove double quotes
    w = re.sub(r"[^a-zA-Z?.!,]+", " ", w) # Remove characters that are not letters, numbers, or punctuation
    w = w.rstrip().strip() # Remove leading and trailing spaces
    w = '<start> ' + w + ' <end>' # Add start and end tokens
    return w

In [6]:
# Example of preprocessing an English sentence
en_sentence = "I have some friends to help."
preprocess_sentence(en_sentence)

In [7]:
# Example of preprocessing a Spanish sentence
sp_sentence = "Aún no he dicho nada."
preprocess_sentence(sp_sentence)

In [8]:
def create_dataset(path, num_examples): # Function to create the dataset
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n') # Read and split lines from file
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')] for l in lines[:num_examples]] # Preprocess each line and split into English-Spanish pairs
    return zip(*word_pairs)

In [9]:
print(create_dataset(path_to_file, 2)) # Print the first 2 pairs of preprocessed sentences

In [10]:
en, sp = create_dataset(path_to_file, None) # Create the dataset

In [11]:
len(en) # Check the number of English sentences in the dataset

In [12]:
def max_length(tensor): # Function to find the maximum length of a sentence in a tensor
    return max(len(t) for t in tensor)

In [13]:
def tokenize(lang): # Function to tokenize a language
    lang_tokenizer = keras.preprocessing.text.Tokenizer(filters='') # Create a tokenizer with no filters
    lang_tokenizer.fit_on_texts(lang) # Fit tokenizer on language
    tensor = lang_tokenizer.texts_to_sequences(lang) # Convert sentences to sequences
    tensor = keras.preprocessing.sequence.pad_sequences(tensor, padding='post') # Pad sequences to the same length
    return tensor, lang_tokenizer

In [14]:
def load_dataset(path, num_examples=None): # Function to load and preprocess the dataset
    targ_lang, inp_lang = create_dataset(path, num_examples) # Create dataset from file
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang) # Tokenize input language
    target_tensor, target_lang_tokenizer = tokenize(targ_lang) # Tokenize target language
    return input_tensor, target_tensor, inp_lang_tokenizer, target_lang_tokenizer

In [15]:
input_tensor, target_tensor, input_lang_tokenizer, target_lang_tokenizer = load_dataset(path_to_file, 20000) # Load and preprocess the dataset

In [16]:
# Check the shape of the input tensor
input_tensor

In [17]:
# Find the maximum length of sentences in the input and target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [18]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(input_tensor, target_tensor, test_size=0.2)

In [19]:
def convert(lang, tensor): # Function to convert tensors into human-readable sentences
    for t in tensor:
        if t != 0:
            print(t, ' .... ', lang.index_word[t])

In [20]:
# Convert the first sentence from the input tensor to text
convert(input_lang_tokenizer, input_tensor[0])

In [21]:
# Define the buffer size and batch size for training
BUFFER_SIZE = len(X_train)
BATCH_SIZE = 64
steps_per_epoch = len(X_train) // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(input_lang_tokenizer.word_index) + 1
vocab_targ_size = len(target_lang_tokenizer.word_index) + 1

In [22]:
# Create a TensorFlow dataset from the training data
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True) # drop_remainder = delete extra characters

In [23]:
class Encoder(keras.Model): # Define the Encoder class
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True)
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state
    def initilize_hidden_state(self):
        return tf.zeros((self.batch_size, self.enc_units))

In [24]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

In [25]:
# Initialize the hidden state of the encoder
simple_hidden = encoder.initilize_hidden_state()
simple_hidden

In [26]:
# Obtain sample output and hidden state from the encoder
example_input_batch, example_target_batch = next(iter(dataset))
simple_output, simple_states = encoder(example_input_batch, simple_hidden)

In [27]:
class Attention(keras.layers.Layer): # Define the Attention class
    def __init__ (self, units):
        super(Attention, self).__init__()
        self.W1 = keras.layers.Dense(units)
        self.W2 = keras.layers.Dense(units)
        self.V = keras.layers.Dense(1)
    def call (self, query, values):
        hidden_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
        atteion_weights = tf.nn.softmax(score, axis=1)
        context_vector = atteion_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, atteion_weights

In [29]:
attention_layer = Attention(10)
attention_layer(simple_hidden, simple_output)

In [30]:
class Decoder(keras.Model): # Define the Decoder class
    def __init__ (self, vocab_size, embedding_dim, dec_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True)
        self.fc = keras.layers.Dense(vocab_size)
        self.attention = Attention(self.dec_units)
    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights

In [31]:
decoder = Decoder(vocab_targ_size, embedding_dim, units, BATCH_SIZE)

In [32]:
# Example of calling the decoder
decoder(tf.random.uniform((BATCH_SIZE, 1)), simple_hidden, simple_output)

In [33]:
optimizer = keras.optimizers.Adam()
loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [34]:
def loss_function(real, pred): # Define the loss function
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [35]:
checkpoint_dir = 'chckpnts'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [36]:
def train_step(inp, targ, enc_hidden): # Define a single training step
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

In [37]:
EPOCHS = 10
for epoch in range(EPOCHS): # Training loop
    enc_hidden = encoder.initilize_hidden_state()
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        print('Epoch: ', epoch)
        print('Loss: ', batch_loss.numpy())
    checkpoint.save(file_prefix='test1')

In [38]:
def evaluate(sentence): # Function to evaluate a sentence
    sentence = preprocess_sentence(sentence)
    inputs = [input_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post') # Pad sequences to the same length
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']], 0)
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        attention_weights = tf.reshape(attention_weights, (-1, ))
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += target_lang_tokenizer.index_word[predicted_id] + ' '
        if target_lang_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence

# Restore the latest checkpoint
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

# Example of using the trained model to translate a sentence
result, sentence = evaluate('hace mucho frio aqui.')
result